# End-effector trajectories

Until now, we have seen how we describe a robotic system in a static configuration, either when it is about the relationship between joint position and end-effector position, or when it is about joint velocity and end-effector velocity (but solved for a specific joint configuration). However, one of the most relevant tasks for a robotic system is to be able to plan its trajectory over time. This is important for two reasons: a) it gives us a way to control the velocity of the robot along a specific path b) it allows us to develop obstacle avoidance algorithms.

There are several methods for executing end-effector trajectories. In this laboratory we will explore a couple of them, starting from the most naive to more advanced ones.

# Single joint trajectories

A trajectory is a set of poses (path) with a time component. Remember that a pose defines both the position and the orientation of a body in space. For now, we will deal with position and orientation together, but later on we will see that we need to do different calculations for each component of pose.

Since we want to control the position of the end-effector, we need to first have calculated the inverse kinematics model of the robot. If we know the inverse kinematics model, then we can control the joint coordinates so that the end-effector moves along the points of the trajectory.

\begin{equation}
    f(P) \rightarrow q
\end{equation}

Suppose that we want to move the end-effector of a robot to move from pose S to point E in space. We would need to firstly calculate the joint coordinates for the start and end position, using the inverse kinematics

\begin{equation}
    f(P_{s}) \rightarrow q_{s}\\
    f(P_{e}) \rightarrow q_{e}
\end{equation}

Remember that $q_s$ and $q_e$ are vectors, having one element for each joint. Therefore, if we want to move the from pose S to pose E, we need to change the joint coordinates from values $q_s$ to the values of $q_e$. There are two main ways of doing this, either using the maximum velocity of each joint, or using the mimimum velocity of all the joints.

## Interpolation basics

In this case, each joint moves individually with its maximum velocity. This will ensure the fastest possible transition from pose S to pose E, as each joint will move to its target coordinate the fastest possible. This means that some joints might finish executing their motion faster than others, based on their maximum velocity and how much distance they need to cover.

\begin{equation}
    q_{1s} \rightarrow q_{1e}\\
    q_{2s} \rightarrow q_{2e}\\
    \vdots \\
    q_{ns} \rightarrow q_{ne}
\end{equation}

The process of changing smoothly a variable from an initial value to a final value over time, is called _interpolation_ and it can be achieved using the following equation:

\begin{equation}
  q(t) = (1-s(t))q_s+s(t)q_e
\end{equation}

Where $q(t)$ is the interpolated value of the joint coordinate over time, $q_s$ and $q_e$ are the starting and ending joint coordinates, and $s(t)$ is a function that varies from 0 to 1 monotonously over time. This means that $s(0) = 0$, $s(T) = 1$, and it has values between 0 and 1 for the time in between. Based on the form of $s(t)$ we can achieve different types of interpolations.

### Linear interpolation

The most basic form of interpolation is _linear interpolation_ and can be achieved if $s(t)$ is a linear function. For example, if want to achieve a motion from $q_s = 1$ to $q_e = 5$ in 8 seconds, then $s(t) = \frac{1}{8}t$, since $s(0) = 0$ and $s(8) = 1$. This means that:

\begin{equation}
    q(0) = (1 - 0)q_s + 0q_e = q_s\\
    q(8) = (1 - 1)q_s + 1q_e = q_e
\end{equation}

The general equation for defining $s(t)$ would be:

\begin{equation}
    s(t) = \dfrac{t-t_s}{t_e-t_s}
\end{equation}

Where $t_s$ and $t_e$ are the starting and ending time respectively. You can see a graphic representation of how the variable s and q look like over time, for various values of time duration, and starting and ending joint coordinates.

In [1]:
from lab_functions import *

def linear_interpolation(qs=-90, qe=0, T=8):
    steps = 20*T
    time = np.linspace(0.01,T,steps)
    s = time/T
    ds = s/time
    dds = ds-ds
    q = (1-s)*qs+s*qe
    dq = (1-ds)*qs+ds*qe
    ddq = (1-dds)*qs+dds*qe
    s.shape = (steps,1)
    q.shape = (steps,1)
    dq.shape = (steps,1)
    ddq.shape = (steps,1)
    plot_interpolation(time, s, q, dq, ddq, 'Linear interpolation', False)
    
    zers = np.asmatrix(np.zeros((time.shape[0], 1)))
    qs = np.concatenate((zers, q, zers, zers, zers, zers), axis=1)
    robot = rtb.models.URDF.UR5()
    robot.plot(qs*3.131/180, "swift")
    
interact_manual(linear_interpolation, qs=(-90,90), qe=(-90,90), T=(1,10))

interactive(children=(IntSlider(value=-90, description='qs', max=90, min=-90), IntSlider(value=0, description=…

<function __main__.linear_interpolation(qs=-90, qe=0, T=8)>

The main disadvantage of linear interpolation is that it introduces discontiniouities in the first and second derivatives of the variable. This means that velocity and acceleration will need to change instanteounsly, which will introduce trouble for the controller.

### Polynomial interpolation

To alleviate the problems of discontiniouities in the velocity and acceleration, we can perform more complex forms of interpolation, namely with a polynomial instead of a linear function. We can choose the order of the polynomial to our needs, but if we want to ensure continiouity in the first and second derivaties at the beginning and the end of the trajectory, then we need at least a 5th order polynomial. This polynomial would look like this:

\begin{equation}
  s(t) = At^5 + Bt^4 + Ct^3 + Dt^2 + Et + F, t \in [0,T]
\end{equation}

To ensure continiouity, we can define the following boundary conditions, that define the six coefficients of the polynomial.

\begin{equation}
  s(0) = q_s \\
  s(T) = q_e \\ 
  \dot{s}(0) = 0 \\
  \dot{s}(T) = 0 \\
  \ddot{s}(0) = 0 \\
  \ddot{s}(T) = 0 \\
\end{equation}

Knowning that:

\begin{equation}
  \dot{s} = 5As^4 + 4Bs^3 + 3Cs^2 + 2Ds + E \\
  \ddot{s} = 20As^3 + 12Bs^2 + 6Cs + 2D
\end{equation}

We can of course define different starting and ending velocities and accelerations. Using these boundary conditions, we end up with a system of six equations with six unknowns, that can be expressed in algebraic form like this:

\begin{equation}
\left[ {\begin{array}{*{20}c}
   s(0)  \\
   s(T)  \\
   \dot{s}(0)\\
   \dot{s}(T)\\
   \ddot{s}(0)\\
   \ddot{s}(T)\\
\end{array}}
\right] = \left[ {\begin{array}{*{20}c}
   0   & 0   & 0   & 0   & 0  & 1 \\
   T^5 & T^4 & T^3 & T^2 & T  & 1 \\
   0   & 0   & 0   & 0   & 1  & 0 \\
   5T^4& 4T^3& 3T^2& 2T  & 1  & 0 \\
   0   & 0   & 0   & 2   & 0  & 0 \\
   20T^3&12T^2&6T& 2  & 0  & 0 \\
\end{array}} \right] \left[ {\begin{array}{*{20}c}
   A  \\
   B  \\
   C  \\
   D  \\
   E  \\
   F  \\
\end{array}}
\right]
\end{equation}

This is a linear system of equations that can be easily solved by inverting the 6x6 matrix. Remember that we are looking for the coefficiencts A, B, C, D, E, and F for desired boundary conditions (velocity and acceleration at the beginning and the end of the trajectory). You can see a graphic representation of how the variable s and q look like over time, for various values of time duration, starting and ending joint coordinates, and boundary conditions.

In [2]:
from lab_functions import *

def polynomial_interpolation(qs=-90, qe=-20, dss=0, dse=0, ddss=0, ddse=0, T=8):
    steps = 20*T
    time = np.linspace(0,T,steps)
    A = np.array([[0,0,0,0,0,1],[T**5, T**4, T**3,T**2, T, 1],[0,0,0,0,1,0],[5*T**4,4*T**3,3*T**2,2*T,1,0],[0,0,0,2,0,0],[20*T**3,12*T**2,6*T,2,0,0]])
    inpt = np.array([0,1,dss,dse,ddss,ddse])
    inpt.shape = (6,1)
    sol = np.linalg.inv(A).dot(inpt)
    timed = np.array((np.power(time,5),np.power(time,4),np.power(time,3),np.power(time,2),time,np.ones((steps,))))
    s = sol[0]*timed[0]+sol[1]*timed[1]+sol[2]*timed[2]+sol[3]*timed[3]+sol[4]*timed[4]+sol[5]*timed[5]
    ds = 5*sol[0]*timed[1]+4*sol[1]*timed[2]+3*sol[2]*timed[3]+2*sol[3]*timed[4]+sol[4]
    dds = 20*sol[0]*timed[2]+12*sol[1]*timed[3]+6*sol[2]*timed[4]+2*sol[3]
    q = (1-s)*qs+s*qe
    dq = (1-ds)*qs+ds*qe
    ddq = (1-dds)*qs+dds*qe
    s.shape = (steps,1)
    q.shape = (steps,1)
    dq.shape = (steps,1)
    ddq.shape = (steps,1)
    plot_interpolation(time, s, q, dq, ddq, 'Polynomial interpolation', True)
    
    zers = np.asmatrix(np.zeros((time.shape[0], 1)))
    qs = np.concatenate((zers, q, zers, zers, zers, zers), axis=1)
    robot = rtb.models.URDF.UR5()
    robot.plot(qs*3.131/180, "swift")
    
interact_manual(polynomial_interpolation, qs=(-90,90), qe=(-90,90), dss=(-2,2,0.1), dse=(-2,2,0.1), ddss=(-2,2,0.01), ddse=(-2,2,0.01), T=(1,10))

interactive(children=(IntSlider(value=-90, description='qs', max=90, min=-90), IntSlider(value=-20, descriptio…

<function __main__.polynomial_interpolation(qs=-90, qe=-20, dss=0, dse=0, ddss=0, ddse=0, T=8)>

Obviously, we can do the same thing for two or more joints. We just need to decide on the boundary conditions of each joint and calculate its own trajectory

In [3]:
from lab_functions import *

def polynomial_interpolation(qs=0, qe=70, dss=0, dse=0, ddss=0, ddse=0, T=8):
    steps = 20*T
    time = np.linspace(0,T,steps)
    timed = np.array((np.power(time,5),np.power(time,4),np.power(time,3),np.power(time,2),time,np.ones((steps,))))
    A = np.array([[0,0,0,0,0,1],[T**5, T**4, T**3,T**2, T, 1],[0,0,0,0,1,0],[5*T**4,4*T**3,3*T**2,2*T,1,0],[0,0,0,2,0,0],[20*T**3,12*T**2,6*T,2,0,0]])
    inpt1 = np.array([0,1,0,0,0,0])
    inpt1.shape = (6,1)
    sol1 = np.linalg.inv(A).dot(inpt1)
    s1 = sol1[0]*timed[0]+sol1[1]*timed[1]+sol1[2]*timed[2]+sol1[3]*timed[3]+sol1[4]*timed[4]+sol1[5]*timed[5]
    q1 = (1-s1)*-90-s1*20
    
    inpt = np.array([0,1,dss,dse,ddss,ddse])
    inpt.shape = (6,1)
    sol = np.linalg.inv(A).dot(inpt)
    s = sol[0]*timed[0]+sol[1]*timed[1]+sol[2]*timed[2]+sol[3]*timed[3]+sol[4]*timed[4]+sol[5]*timed[5]
    q = (1-s)*qs+s*qe
    
    q.shape = (steps,1)
    q1.shape = (steps,1)
    
    zers = np.asmatrix(np.zeros((time.shape[0], 1)))
    qs = np.concatenate((zers, q1, -q, zers, zers, zers), axis=1)
    print(qs)
    robot = rtb.models.URDF.UR5()
    robot.plot(qs*3.131/180, "swift")
    
interact_manual(polynomial_interpolation, qs=(-90,90), qe=(-90,90), dss=(-2,2,0.1), dse=(-2,2,0.1), ddss=(-2,2,0.01), ddse=(-2,2,0.01), T=(1,10))

interactive(children=(IntSlider(value=0, description='qs', max=90, min=-90), IntSlider(value=70, description='…

<function __main__.polynomial_interpolation(qs=0, qe=70, dss=0, dse=0, ddss=0, ddse=0, T=8)>

# End effector trajectories

Controlling each individual joint is not by itself very useful, as we are usually interested to control the end-effector pose. As we know from inverse kinematics we can calculate the joint coordinates required to achieve a certain pose, depending always on the degrees of freedom of our robotic arm. Therefore, if we can calculate a trajectory for the end-effector, then we can calculate the required joint coordinates for achieving the trajectory using inverse kinematics. In order to produce once again smooth transitions in pose, we will need to interpolate from a starting to an ending position. We will decompose the interpolation of pose into two parts: position and orientation

## Interpolating position

Interpolating the position is rather straight-forward, as we can interpolate each one of the components of the position vector ($X$, $Y$, and $Z$ coordinates) from the beginning to the end-position. If we want to ensure smooth motion without jerk, we should once again apply a polynomial interpolation to each coordinate individually.

\begin{equation}
  P_x(0) \rightarrow P_x(T)\\
  P_y(0) \rightarrow P_y(T)\\
  P_z(0) \rightarrow P_z(T)\\
\end{equation}

In [4]:
from lab_functions import *

def threed_interpolation(xs=0, xe=20, ys=0, ye=30, zs=0, ze=10, T=8):
    dss, dse, ddss, ddse = 0,0,0,0
    steps = 20*T
    time = np.linspace(0,T,steps)
    timed = np.array((np.power(time,5),np.power(time,4),np.power(time,3),np.power(time,2),time,np.ones((steps,))))
    A = np.array([[0,0,0,0,0,1],[T**5, T**4, T**3,T**2, T, 1],[0,0,0,0,1,0],[5*T**4,4*T**3,3*T**2,2*T,1,0],[0,0,0,2,0,0],[20*T**3,12*T**2,6*T,2,0,0]])  
    inpt = np.array([0,1,dss,dse,ddss,ddse])
    inpt.shape = (6,1)
    sol = np.linalg.inv(A).dot(inpt)
    s = sol[0]*timed[0]+sol[1]*timed[1]+sol[2]*timed[2]+sol[3]*timed[3]+sol[4]*timed[4]+sol[5]*timed[5]
    x = (1-s)*xs+s*xe
    y = (1-s)*ys+s*ye
    z = (1-s)*zs+s*ze
    
    x.shape = (steps,1)
    y.shape = (steps,1)
    z.shape = (steps,1)
    
    plot_car_interpolation(time, x, y, z, '3D interpolation')
    
interact_manual(threed_interpolation, xs=(0,90), xe=(-90,90), ys=(-2,2,0.1), ye=(-2,2,0.1), zs=(-2,2,0.01), ze=(-2,2,0.01), T=(1,10))

interactive(children=(IntSlider(value=0, description='xs', max=90), IntSlider(value=20, description='xe', max=…

<function __main__.threed_interpolation(xs=0, xe=20, ys=0, ye=30, zs=0, ze=10, T=8)>

Now that we have a cartesian trajectory, we can generate the joint trajectories by solving the inverse kinematics model for each step of the trajectory.

In [5]:
from lab_functions import *

def eef_translation(xs=0.6, xe=0.5, ys=0.2, ye=0.4, zs=0.3, ze=0.4, T=8):
    dss, dse, ddss, ddse = 0,0,0,0
    steps = 20*T
    time = np.linspace(0,T,steps)
    timed = np.array((np.power(time,5),np.power(time,4),np.power(time,3),np.power(time,2),time,np.ones((steps,))))
    A = np.array([[0,0,0,0,0,1],[T**5, T**4, T**3,T**2, T, 1],[0,0,0,0,1,0],[5*T**4,4*T**3,3*T**2,2*T,1,0],[0,0,0,2,0,0],[20*T**3,12*T**2,6*T,2,0,0]])  
    inpt = np.array([0,1,dss,dse,ddss,ddse])
    inpt.shape = (6,1)
    sol = np.linalg.inv(A).dot(inpt)
    s = sol[0]*timed[0]+sol[1]*timed[1]+sol[2]*timed[2]+sol[3]*timed[3]+sol[4]*timed[4]+sol[5]*timed[5]
    x = (1-s)*xs+s*xe
    y = (1-s)*ys+s*ye
    z = (1-s)*zs+s*ze
    
    x.shape = (steps,1)
    y.shape = (steps,1)
    z.shape = (steps,1)
    
    robot = rtb.models.URDF.UR5()
    qs = np.zeros((steps,6))
    for i in range(steps):
        setpoint = SE3(x[i,0],y[i,0],z[i,0])*SE3.Rx(180,'deg')*SE3.Rz(-90,'deg')
        if i == 0:
            qs[i,:] = robot.ikine_LM(setpoint).q
        else:
            qs[i,:] = robot.ikine_LM(setpoint, q0=qs[max(i-1,0),:]).q

    robot.plot(qs, "swift")
    
    plot_car_interpolation(time, x, y, z, 'Cartesian space interpolation')
    plot_joint_interpolation(time, qs[:,0], qs[:,1], qs[:,2], qs[:,3], qs[:,4], qs[:,5], 'Joint space interpolation')

interact_manual(eef_translation, xs=(-0.9,0.9), xe=(-0.9,0.9), ys=(-0.9,0.9), ye=(-0.9,0.9), zs=(-0.9,0.9), ze=(-0.9,0.9), T=(1,10))

interactive(children=(FloatSlider(value=0.6, description='xs', max=0.9, min=-0.9), FloatSlider(value=0.5, desc…

<function __main__.eef_translation(xs=0.6, xe=0.5, ys=0.2, ye=0.4, zs=0.3, ze=0.4, T=8)>

## Interpolating orientation

Orientation is more complicated to interpolate, as its components cannot be interpolated independently of each other when written in a matrix form. This is because the numbers of the transformation matrix are not all independent from each other, and each column represents a unit vector. As such, we cannot interpolate the numbers from our starting orientation toward the ending orientation, since the interpolated values are not guaranteed to form unit vectors. Consider the following simple example of interpolating from one unit vector to another:

\begin{equation}
  \left[ {\begin{array}{*{20}c}
   0  \\
   0  \\
   1
\end{array}}
\right] \rightarrow
\left[ {\begin{array}{*{20}c}
   1  \\
   0  \\
   0
\end{array}}
\right]
\end{equation}

if we interpolate each component individually, at some point we will end up with vector:

\begin{equation}
\left[ {\begin{array}{*{20}c}
   0.5  \\
   0  \\
   0.5
\end{array}}
\right]
\end{equation}

which is not a unit vector!

Therefore, for interpolating orientation, we need to break the matrix down to its components. As we have seen in the transformations laboratory, there are different ways to express orientation. If we use the Euler angle representation, then we can interpolate the components individually and end up with a smooth transition from one orientation to another one.

In [6]:
from lab_functions import *

def eef_pose(rs=145, re=180, ps=110, pe=40, yas=50, yae=90, xs=0.2, xe=0.4, ys=0.2, ye=0.4, zs=0.2, ze=0.4, T=8):
    dss, dse, ddss, ddse = 0,0,0,0
    steps = 20*T
    time = np.linspace(0,T,steps)
    timed = np.array((np.power(time,5),np.power(time,4),np.power(time,3),np.power(time,2),time,np.ones((steps,))))
    A = np.array([[0,0,0,0,0,1],[T**5, T**4, T**3,T**2, T, 1],[0,0,0,0,1,0],[5*T**4,4*T**3,3*T**2,2*T,1,0],[0,0,0,2,0,0],[20*T**3,12*T**2,6*T,2,0,0]])  
    inpt = np.array([0,1,dss,dse,ddss,ddse])
    inpt.shape = (6,1)
    sol = np.linalg.inv(A).dot(inpt)
    s = sol[0]*timed[0]+sol[1]*timed[1]+sol[2]*timed[2]+sol[3]*timed[3]+sol[4]*timed[4]+sol[5]*timed[5]
    x = (1-s)*xs+s*xe
    y = (1-s)*ys+s*ye
    z = (1-s)*zs+s*ze
    r = ((1-s)*rs+s*re)
    p = ((1-s)*ps+s*pe)
    ya = ((1-s)*yas+s*yae)
    
    x.shape = (steps,1)
    y.shape = (steps,1)
    z.shape = (steps,1)
    r.shape = (steps,1)
    p.shape = (steps,1)
    ya.shape = (steps,1)
    
    robot = rtb.models.URDF.UR5()
    qs = np.zeros((steps,6))
    for i in range(steps):
        setpoint = SE3(x[i,0],y[i,0],z[i,0])*SE3.RPY([r[i,0],p[i,0],ya[i,0]], unit='deg')
        qs[i,:] = robot.ikine_LM(setpoint, q0=qs[max(i-1,0),:]).q

    robot.plot(qs, "swift")
    
    plot_car_interpolation(time, x, y, z, 'Cartesian space interpolation')
    plot_joint_interpolation(time, qs[:,0], qs[:,1], qs[:,2], qs[:,3], qs[:,4], qs[:,5], 'Joint space interpolation')

interact_manual(eef_pose, rs=(-180,180), re=(-180,180), ps=(-180,180), pe=(-180,180), yas=(-180,180), yae=(-180,180), xs=(-0.9,0.9), xe=(-0.9,0.9), ys=(-0.9,0.9), ye=(-0.9,0.9), zs=(-0.9,0.9), ze=(-0.9,0.9), T=(1,10))

interactive(children=(IntSlider(value=145, description='rs', max=180, min=-180), IntSlider(value=180, descript…

<function __main__.eef_pose(rs=145, re=180, ps=110, pe=40, yas=50, yae=90, xs=0.2, xe=0.4, ys=0.2, ye=0.4, zs=0.2, ze=0.4, T=8)>

Similarly, we can use quaternions as well, which can be interpolated as well. Using quaternions avoids problems with 'gimbal lock' (loss of one or more degrees of freedom), and produce a shorter path interpolation than Euler angles.

Quaternions can be interpolated using the [SLERP](https://en.wikipedia.org/wiki/Slerp) method (**S**pherical **L**inear int**ERP**olation), which is implemented in the [SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Slerp.html) library of Python. The procedure for performing the interpolation is the same as with the previous cases: Choose initial and final orientation, calculate the interpolation polynomial, interpolate the value of the angles at the timesteps of the polynomial (using SLERP).

Below, you can see a basic example of how to use SLERP for interpolating between two orientations using SLERP.

In [7]:
from scipy.spatial.transform import Slerp
from lab_functions import *

def quat_vs_eul(rs=180, re=-20, ys=20, ye=180, T=8, quaternions=True):
    # It is easier if we create a quaternion from a known transformation matrix (or euler angles).
    init_q = r2q(rotz(ys, 'deg')@rotx(rs, 'deg'))
    final_q = r2q(rotz(ye, 'deg')@rotx(re, 'deg'))

    # Constructing interpolator s using polynomial
    T = 8 # Total time
    dss, dse, ddss, ddse = 0,0,0,0 # Starting and ending velocity and acceleration
    steps = 20*T
    time = np.linspace(0,T,steps)
    timed = np.array((np.power(time,5),np.power(time,4),np.power(time,3),np.power(time,2),time,np.ones((steps,))))
    A = np.array([[0,0,0,0,0,1],[T**5, T**4, T**3,T**2, T, 1],[0,0,0,0,1,0],[5*T**4,4*T**3,3*T**2,2*T,1,0],[0,0,0,2,0,0],[20*T**3,12*T**2,6*T,2,0,0]])  
    inpt = np.array([0,1,dss,dse,ddss,ddse]).reshape(6,1)
    sol = np.linalg.inv(A).dot(inpt)
    s = sol[0]*timed[0]+sol[1]*timed[1]+sol[2]*timed[2]+sol[3]*timed[3]+sol[4]*timed[4]+sol[5]*timed[5]

    quat = np.zeros((4,steps))
    # Interpolating in Quaternion space
    for i in range(steps):
        quat[:,i] = qslerp(init_q, final_q, s=s[i], shortest=True)
    
    # Interpolating in Euler space
    r = ((1-s)*rs+s*re)
    y = ((1-s)*ys+s*ye)
    
    robot = rtb.models.URDF.UR5()
    qs = np.zeros((steps,6))
    for i in range(steps):
        if quaternions:
            setpoint = SE3(0.1,0.2,0.5)*SE3.Rt(q2r(quat[:,i]))
        else:
            setpoint = SE3(0.1,0.2,0.5)*SE3.RPY([r[i],0,y[i]], unit='deg', order='zyx')
        qs[i,:] = robot.ikine_LM(setpoint, q0=qs[max(i-1,0),:]).q
    
    robot.plot(qs, "swift")
    
interact_manual(quat_vs_eul, rs=(-180,180), re=(-180,180), ys=(-180,180), ye=(-180,180), T=(1,10))

interactive(children=(IntSlider(value=180, description='rs', max=180, min=-180), IntSlider(value=-20, descript…

<function __main__.quat_vs_eul(rs=180, re=-20, ys=20, ye=180, T=8, quaternions=True)>

# Robotics Toolbox

The toolbox allows us to construct trajectories between two poses, using the __ctraj__ method. Simply give as input the initial and final pose, and the number of steps in between, and the toolbox will calculate the poses in between. If we then need to move our robot along the desired trajectory, we need to solve the inverse kinematics for each individual pose calculated. In order to minimize the time required, for each call of ikine, we can give the solution of the previous step as an initial guess for the numerical optimization. This is what is demonstrated in the example below

In [ ]:
from lab_functions import *

# We define starting and ending poses
T1  = SE3(0.3,0.2,0.3)@SE3.RPY(180,0,90, unit='deg')
T2  = SE3(0.5,0.3,0.5)@SE3.RPY(110,20,70, unit='deg')
steps = 100
tr = rtb.ctraj(T1,T2,steps) # tr is a collection of cartesian poses of size 'steps'

# We load our robot from the toolbox
robot = rtb.models.URDF.UR5()

# We solve the inverse kinematics for the target poses
sol = robot.ikine_LM(tr)

# Animating the trajectory
robot.plot(sol.q)

# Assignment

## AL5D robot

Using the code you constructed for the simulation above, implement a trajectory following for the AL5D robot for the following starting and ending poses
    
|Coord|Initial|Final|
|-|-|-|
|<td colspan=3>Position<td colspan=2>
|X|0.16|0.18|
|Y|0.22|-0.18|
|Z|0.09|0.13|
|<td colspan=3>Orientation (Euler 'zyx' angles) <td colspan=2>
|roll|0|0|
|pitch|20|-90|
|yaw|0|10|
    
Compare the results between Euler and Quaternion interpolation

In [9]:
from lab_functions import *
from al5d_control import *
from time import sleep

# for sending the commands
rrob = AL5DControl()

rs = 0
re = 0
ps = -20
pe = -90
yas = 0
yae = 10
xs = 0.16
xe = 0.18
ys = 0.22
ye = -0.18
zs = 0.09
ze = 0.13
T = 8

dss, dse, ddss, ddse = 0,0,0,0
steps = 20*T
time = np.linspace(0,T,steps)
timed = np.array((np.power(time,5),np.power(time,4),np.power(time,3),np.power(time,2),time,np.ones((steps,))))
A = np.array([[0,0,0,0,0,1],[T**5, T**4, T**3,T**2, T, 1],[0,0,0,0,1,0],[5*T**4,4*T**3,3*T**2,2*T,1,0],[0,0,0,2,0,0],[20*T**3,12*T**2,6*T,2,0,0]])  
inpt = np.array([0,1,dss,dse,ddss,ddse])
inpt.shape = (6,1)
sol = np.linalg.inv(A).dot(inpt)
s = sol[0]*timed[0]+sol[1]*timed[1]+sol[2]*timed[2]+sol[3]*timed[3]+sol[4]*timed[4]+sol[5]*timed[5]

x = (1-s)*xs+s*xe
y = (1-s)*ys+s*ye
z = (1-s)*zs+s*ze
r = ((1-s)*rs+s*re)
p = ((1-s)*ps+s*pe)
ya = ((1-s)*yas+s*yae)
    
x.shape = (steps,1)
y.shape = (steps,1)
z.shape = (steps,1)
r.shape = (steps,1)
p.shape = (steps,1)
ya.shape = (steps,1)

No device connected on  /dev/ttyUSB0
No device connected on  /dev/ttyUSB1
No device connected on  COM1
No device connected on  COM2
No device connected on  COM3
No device connected on  COM4
No device connected on  COM5
No device connected on  COM6
No device connected on  COM7
No device connected on  COM8
No device connected on  COM9
Robot could not connect to any ports, restart kernel and try again


In [10]:
# Interpolating in Euler space
robot = rtb.models.URDF.AL5D_mdw()
    
qs1 = np.zeros((steps,5))
for i in range(steps):
    setpoint = SE3(x[i,0],y[i,0],z[i,0])*SE3.RPY([r[i,0],p[i,0],ya[i,0]], unit='deg')
    qs1[i,:] = robot.ikine_LM(setpoint, q0=qs1[max(i-1,0),:], mask=np.array([1,1,1,0,1,1])).q

robot.plot(qs1, "swift")

Swift backend, t = 7.99999999999998, scene:
  AL5D_mdw

In [11]:
# Interpolating in Quaternion space
robot = rtb.models.URDF.AL5D_mdw()

init_q = r2q(rotz(yas, 'deg')@roty(ps, 'deg')@rotx(rs, 'deg'))
final_q = r2q(rotz(yae, 'deg')@roty(pe, 'deg')@rotx(re, 'deg'))

quat = np.zeros((4,steps))
for i in range(steps):
    quat[:,i] = qslerp(init_q, final_q, s=s[i], shortest=True)
    
qs2 = np.zeros((steps,5))
for i in range(steps):
    setpoint = SE3(x[i,0],y[i,0],z[i,0])*SE3.Rt(q2r(quat[:,i]))
    qs2[i,:] = robot.ikine_LM(setpoint, q0=qs2[max(i-1,0),:], mask=np.array([1,1,1,0,1,1])).q
    
robot.plot(qs2, "swift")

Swift backend, t = 7.99999999999998, scene:
  AL5D_mdw

In [ ]:
# Solve the ikine for the respective interpolated poses
# The solution will be in radians!

# Control the robot to go to each pose using the robot_control function
# Give degrees as input for the joint values
for i in range(steps):
    rrob.robot_control(*np.rad2deg(qs[i,0]),*np.rad2deg(qs[i,1]),*np.rad2deg(qs[i,2]),*np.rad2deg(qs[i,3]),*np.rad2deg(qs[i,4]),*np.rad2deg(qs[i,5]),0)
    time.sleep(0.1)